# Segmenting and Clustering Neighborhoods in Toronto

This notebook serves as my solution the Applied Data Science Capstone - Week 3 peer graded assignment.

First thing to do is to read the webpage. Using the `BeautifulSoup` package, I'll parse the webpage into `lxml`.

In [1]:
import urllib.request
from bs4 import BeautifulSoup


url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")

Looking at the Wikipedia page, there's only one table in it and its class attribute is `wikitable sortable`. Using that information and the `find` function of the `BeautifulSoup` package, we find the table xml

In [2]:
postal_code_table = soup.find('table', class_='wikitable sortable')

Knowing how HTML tables are written, I looped through each `tr` of the HTML table and use follow the instructions:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [3]:
PostalCode = []
Borough = []
Neighborhood = []

for row in postal_code_table.findAll('tr')[1:]:
    cells = row.findAll('td')
    borough = cells[1].find(text=True).strip()
    
    if borough != 'Not assigned':
        PostalCode.append(cells[0].find(text=True).strip())
        Borough.append(borough)
        neighborhood = cells[2].find(text=True).strip()
        if neighborhood == 'Not assigned': 
            Neighborhood.append(borough)
        else:
            Neighborhood.append(neighborhood)

In [4]:
import pandas as pd

df = pd.DataFrame(PostalCode,columns=['PostalCode'])
df['Borough'] = Borough
df['Neigborhood'] = Neighborhood

In [5]:
df.shape

(103, 3)

## Getting the latitude and longitude

Tried using the `geocoder` package, but it keeps returning `<[REQUEST_DENIED] Google - Geocode [empty]>`. So I'll be using the geospatial data.

In [15]:
geospatial_df = pd.read_csv('Geospatial_Coordinates.csv')
geospatial_df['PostalCode'] = geospatial_df['Postal Code']
geospatial_df.drop(['Postal Code'], axis=1, inplace=True)

df = pd.merge(df, geospatial_df, on='PostalCode')

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


NameError: name 'geosptial_df' is not defined

In [26]:
df.head()

,PostalCode,Borough,Neigborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
